In [24]:
import warnings
from causalnex.structure import StructureModel
from causalnex.structure.dynotears import from_pandas_dynamic
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from xml.etree.ElementTree import tostring
import pandas as pd
from IPython.display import IFrame, display, HTML
import numpy as np
import re
from PIL import Image
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl

In [25]:
sm = StructureModel()
sm.add_edges_from([
    ("MITM", "UnauthCmd"),
    ("MITM", "SpoofRepMsg"),
    ("SpoofRepMsg", "UnstablePS"),
    ("UnauthCmd", "UnstablePS")
])

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.save_graph("supporting_files/01_simple_plot.html")

In [26]:
# Load & convert df to numeric values
df = pd.read_csv("outTrainBase.csv")

colList = list()
for col in df:
    colList.append(df[col].to_numpy())
colSeries = pd.Series(np.array(colList).flatten()).unique()

for scol in colSeries:
    toReplace = int(re.findall(r'\d\d*', scol)[0])
    df.replace(to_replace={scol: toReplace}, inplace=True)

In [27]:
# TODO Convert df to the way "from_pandas_dynamic" expects it
cols = np.array(df.columns)
nodesList = list()
maxSlice = 0

# Convert to numeric values
for col in cols:
  if not re.match(r'^\D+_\d+$', col):
    nodesList.append(col)
  else:
    slice = int(re.findall(r'\d+', col)[0])
    if slice > maxSlice:
      maxSlice = slice

# Convert to list of dataframes (each dataframe is a simulation run)
dfList = list()
for r in range(len(df)):
  outDf = pd.DataFrame()
  for node in nodesList:
    nodeList = list()
    for i in range(0, 24):
      oldName = ""
      if i == 0:
        oldName = node
      else:
        oldName = node + "_" + str(i)

      nodeList.append(df[oldName][r])
    outDf[node] = pd.Series(nodeList)
    dfList.append(outDf)


In [28]:
print("Started")
m = from_pandas_dynamic(dfList, p=1, lambda_w=.1,lambda_a=.1,w_threshold=.1)
print("Ended")

Started
Ended


In [30]:

viz = plot_structure(
    m,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.save_graph("supporting_files/tresh-0.6.html")

<bound method Graph.adjacency of <causalnex.structure.structuremodel.StructureModel object at 0x7fc43862bf70>>
